First things first. Let us set up the environment with the requried packages for this notebook: `HDF5` is used to load .h5 files, `Plots` will be used to plot images, `Distributions` will be used for the statistical functions, and `Plotly` is a Julia interface to the plot.ly plotting library and cloud services.

In [1]:
for p in ("HDF5", "Plots", "Distributions", "Plotly.jl")
    Pkg.installed(p) == nothing && Pkg.add(p)
end

using HDF5, Plots, Distributions
gr()

Plots.GRBackend()

# Probability and statistics


In some form or another, machine learning is all about making predictions. 
We might want to predict the *probability* of a patient suffering a heart attack in the next year,
given their clinical history.
In anomaly detection, we might want to assess how *likely* a set of readings from an airplane's jet engine would be,
were it operating normally. 
In reinforcement learning, we want an agent to act intelligently in an environment. 
This means we need to think about the probability of getting a high reward under each of the available action. 
And when we build recommender systems we also need to think about probability. 
For example, if we *hypothetically* worked for a large online bookseller,
we might want to estimate the probability that a particular user would buy a particular book, if prompted. 
For this we need to use the language of probability and statistics. 
Entire courses, majors, theses, careers, and even departments, are devoted to probability.
So our goal here isn't to teach the whole subject. 
Instead we hope to get you off the ground,
to teach you just enough that you know everything necessary to start building your first machine learning models 
and to have enough of a flavor for the subject that you can begin to explore it on your own if you wish.


We've talked a lot about probabilities so far without articulating what precisely they are or giving a concrete example. Let's get more serious by considering the problem of distinguishing cats and dogs based on photographs. This might sound simpler but it's actually a formidable challenge. To start with, the difficulty of the problem may depend on the resolution of the image.

| 20px | 40px | 80px | 160px | 320px |
|:----:|:----:|:----:|:-----:|:-----:|
|![](../img/whitecat20.jpg)|![](../img/whitecat40.jpg)|![](../img/whitecat80.jpg)|![](../img/whitecat160.jpg)|![](../img/whitecat320.jpg)|
|![](../img/whitedog20.jpg)|![](../img/whitedog40.jpg)|![](../img/whitedog80.jpg)|![](../img/whitedog160.jpg)|![](../img/whitedog320.jpg)|

While it's easy for humans to recognize cats and dogs at 320 pixel resolution, 
it becomes challenging at 40 pixels 
and next to impossible at 20 pixels. 
In other words, our ability to tell cats and dogs apart at a large distance (and thus low resolution) 
might approach uninformed guessing. 
Probability gives us a formal way of reasoning about our level of certainty.
If we are completely sure that the image depicts a cat, 
we say that the *probability* that the corresponding label $l$ is $\mathrm{cat}$, 
denoted $P(l=\mathrm{cat})$ equals 1.0.
If we had no evidence to suggest that $l =\mathrm{cat}$ or that $l = \mathrm{dog}$,
then we might say that the two possibilities were equally $likely$
expressing this as $P(l=\mathrm{cat}) = 0.5$.
If we were reasonably confident, but not sure that the image depicted a cat,
we might assign a probability $.5  < P(l=\mathrm{cat}) < 1.0$.

Now consider a second case:
given some weather monitoring data,
we want to predict the probability that it will rain in Taipei tomorrow.
If it's summertime, the rain might come with probability $.5$
In both cases, we have some value of interest.
And in both cases we are uncertain about the outcome.
But There's a key difference between the two cases. 
In this first case, the image is in fact either a dog or a cat, 
we just don't know which. 
In the second case, the outcome may actually be a random event,
if you believe in such things (and most physicists do).
So probability is a flexible language for reasoning about our level of certainty,
and it can be applied effectively in a broad set of contexts.

## Basic probability theory

Say that we cast a die and want to know 
what the chance is of seeing a $1$ 
rather than another digit. 
If the die is fair, all six outcomes $\mathcal{X} = \{1, \ldots, 6\}$ 
are equally likely to occur, 
hence we would see a $1$ in $1$ out of $6$ cases. 
Formally we state that $1$ occurs with probability $\frac{1}{6}$. 

For a real die that we receive from a factory,
we might not know those proportions 
and we would need to check whether it is tainted. 
The only way to investigate the die is by casting it many times 
and recording the outcomes. 
For each cast of the die, 
we'll observe a value $\{1, 2, \ldots, 6\}$. 
Given these outcomes, we want to investigate the probability of observing each outcome.

One natural approach for each value is to take the individual count for that value
and to divide it by the total number of tosses.
This gives us an *estimate* of the probability of a given event. 
The law of large numbers tell us that as the number of tosses grows this estimate will draw closer and closer to the true underlying probability.
Before going into the details of what's going here, let's try it out.


Next, we'll want to be able to cast the die.
In statistics we call this process of drawing examples from probability distributions *sampling*.
The distribution which assigns probabilities to a number of discrete choices is called 
the *multinomial* distribution. 
We'll give a more formal definition of *distribution* later,
but at a high level, think of it as just an assignment of probabilities to events.
In MXNet, we can sample from the multinomial distribution via the aptly named `Multinomial` function (which requires the `Distributions.jl` package). To draw say 5 samples, we simply pass in a vector of probabilities:

In [2]:
probabilities = normalize(ones(6), 1);
n = 1 # number of trials with probability vector p
m = 5 # number of randomly drawn samples from distribution
rand(Multinomial(n,probabilities), m)

6×5 Array{Int64,2}:
 0  1  0  0  1
 0  0  0  1  0
 0  0  0  0  0
 0  0  1  0  0
 0  0  0  0  0
 1  0  0  0  0

As you can see, when the distribution $S$ being sampled is multivariative, `rand(S, m)` will return a matrix with $m$ columns (where $m$ is the number of samples). As with estimating the fairness of a die,  we often want to generate many samples from the same distribution. `Multivariative` allows you to instead specify the number of trials in the distribution (i.e. it creates a distribution based on the probability and the number of trials $n$ desired). Let's us visualize this process:

In [6]:
nmax = 1000; # maximum number of trials 
R    = zeros(nmax, 6);
for n in 1:nmax; R[n, :] = rand(Multinomial(n, probabilities))/n; end

In [7]:
p = plot(R)
for die = 1:6; p.series_list[die].d[:label] = @sprintf "Estimated P(die=%d)" die; end
hline!([0.16666], color="black", line=(:dot,4), labels=:True_Probability)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip6702)" style="stroke:#e26f46; stroke-width:1; stroke-opacity

We could also set $n=1$ and randomly sample the distribution ourselves. Here we optimize by using `cumsum` and by broadcasting. For a simpler version please take a look at the original code [here](http://localhost:8004/notebooks/software/mxnet-the-straight-dope/chapter01_crashcourse/probability.ipynb).

In [8]:
p = plot(cumsum(rand(Multinomial(1, probabilities), 1000), 2)'  ./ [1:1000;]) 
for die = 1:6; p.series_list[die].d[:label] = @sprintf "Estimated P(die=%d)" die; end
hline!([0.16666], color="black", line=(:dot,4), labels=:True_Probability)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip6902)" style="stroke:#e26f46; strok

In our example of casting a die, we introduced the notion of a **random variable**. 
A random variable, which we denote here as $X$ can be pretty much any quantity that is not determistic. Random variables are ways to map outcomes of **random processes** to numbers, like fliping a coin, rolling dice, measuring rainfall, etc. Further, a variable that depends on another random variable is itself a random variable. Random variables could take one value among a set of possibilites. 
We denote sets with brackets, e.g., $\{\mathrm{cat}, \mathrm{dog}, \mathrm{rabbit}\}$.
The items contained in the set are called *elements*,
and we can say that an element $x$ is *in* the set S, by writing $x \in S$.
The symbol $\in$ is read as "in" and denotes membership. 
For instance, we could truthfully say $\mathrm{dog} \in \{\mathrm{cat}, \mathrm{dog}, \mathrm{rabbit}\}$.
When dealing with the rolls of die, we are concerned with a variable $X \in \{1, 2, 3, 4, 5, 6\}$. 

Note that there is a subtle difference between discrete random variables, like the sides of a dice, 
and continuous ones, like the weight and the height of a person. 
There's little point in asking whether two people have exactly the same height. 
If we take precise enough measurements you'll find that no two people on the planet have the exact same height. 
In fact, if we take a fine enough measurement, 
you will not have the same height when you wake up and when you go to sleep.
So there's no purpose in asking about the probability 
that some one is $2.00139278291028719210196740527486202$ meters tall. 
The probability is 0.
It makes more sense in this case to ask whether someone's height falls into a given interval, 
say between 1.99 and 2.01 meters. 
In these cases we quantify the likelihood that we see a value as a density. 
The height of exactly 2.0 meters has no probability, but nonzero density. 
Between any two different heights we have nonzero probability.


There are a few important axioms of probability that you'll want to remember:

* For any event $z$, the probability is never negative, i.e. $\Pr(Z=z) \geq 0$.
* For any two events $Z=z$ and $X=x$ the union is no more likely than the sum of the individual events, i.e. $\Pr(Z=z \cup X=x) \leq \Pr(Z=z) + \Pr(X=x)$.
* For any random variable, the probabilities of all the values it can take must sum to 1 $\sum_{i=1}^n P(Z=z_i) = 1$.
* For any two mutually exclusive events $Z=z$ and $X=x$, the probability that either happens is equal to the sum of their individual probabilities that $\Pr(Z=z \cup X=x) = \Pr(Z=z) + \Pr(X=z)$.

## Dealing with multiple random variables

Very often, we'll want consider more than one random variable at a time. 
For instance, we may want to model the relationship between diseases and symptoms.
Given a disease and symptom, say 'flu' and 'cough', 
either may or may not occur in a patient with some probability.
While we hope that the probability of both would be close to zero,
we may want to estimate these probabilities and their relationships to each other
so that we may apply our inferences to effect better medical care.

As a more complicated example, images contain millions of pixels, thus millions of random variables. 
And in many cases images will come with a label, identifying objects in the image.
We can also think of the label as a random variable.
We can even get crazy and think of all the metadata as random variables
such as location, time, aperture, focal length, ISO, focus distance, camera type, etc.
All of these are random variables that occur jointly. 
When we deal with multiple random variables, 
there are several quantities of interest.
The first is called the joint distribution $\Pr(A, B)$. 
Given any elements $a$ and $b$,
the joint distribution lets us answer,
what is the probability that $A=a$ and $B=b$ simulataneously?
It might be clear that for any values $a$ and $b$, $\Pr(A,B) \leq \Pr(A=a)$. 

This has to be the case, since for $A$ and $B$ to happen, 
$A$ has to happen *and* $B$ also has to happen (and vice versa). 
Thus $A,B$ cannot be more likely than $A$ or $B$ individually. 
This brings us to an interesting ratio: $0 \leq \frac{\Pr(A,B)}{\Pr(A)} \leq 1$. 
We call this a **conditional probability** and denote it by $\Pr(B|A)$, 
the probability that $B$ happens, provided that $A$ has happened. 

Using the definition of conditional probabilities, 
we can derive one of the most useful and celebrated equations in statistics - Bayes' theorem. 
It goes as follows: By construction, we have that $\Pr(A, B) = \Pr(B|A) \Pr(A)$. 
By symmetry, this also holds for $\Pr(A,B) = \Pr(A|B) \Pr(B)$. 
Solving for one of the conditional variables we get:
$$\Pr(A|B) = \frac{\Pr(B|A) \Pr(A)}{\Pr(B)}$$

This is very useful if we want to infer one thing from another, 
say cause and effect but we only know the properties in the reverse direction. 
One important operation that we need to make this work is **marginalization**, i.e., 
the operation of determining $\Pr(A)$ and $\Pr(B)$ from $\Pr(A,B)$.
We can see that the probability of seeing $A$ amounts to accounting 
for all possible choices of $B$ and aggregating the joint probabilities over all of them, i.e. 

$$\Pr(A) = \sum_{B'} \Pr(A,B') \text{ and } \Pr(B) = \sum_{A'} \Pr(A',B)$$

A really useful property to check is for **dependence** and **independence**. 
Independence is when the occurrence of one event does not influence the occurrence of the other.
In this case $\Pr(B|A) = \Pr(B)$. Statisticians typically use $A \perp\!\!\!\perp B$ to express this. 
From Bayes Theorem it follows immediately that also $\Pr(A|B) = \Pr(A)$. 
In all other cases we call $A$ and $B$ dependent. 
For instance, two successive rolls of a dice are independent. 
On the other hand, the position of a light switch and the brightness in the room are not 
(they are not perfectly deterministic, though, 
since we could always have a broken lightbulb, power failure, or a broken switch). 

Let's put our skills to the test. 
Assume that a doctor administers an AIDS test to a patient. 
This test is fairly accurate and fails only with 1% probability 
if the patient is healthy by reporting him as diseased, 
and that it never fails to detect HIV if the patient actually has it. 
We use $D$ to indicate the diagnosis and $H$ to denote the HIV status.
Written as a table the outcome $\Pr(D|H)$ looks as follows:

|             | Patient is HIV positive | Patient is HIV negative |
|:------------|------------------------:|------------------------:|
|Test positive| 1 | 0.01 |
|Test negative| 0 | 0.99 |

Note that the column sums are all one (but the row sums aren't), 
since the conditional probability needs to sum up to $1$, just like the probability. 
Let us work out the probability of the patient having AIDS if the test comes back positive. 
Obviously this is going to depend on how common the disease is, since it affects the number of false alarms.
Assume that the population is quite healthy, e.g. $\Pr(\text{HIV positive}) = 0.0015$. 
To apply Bayes Theorem we need to determine 

$$\Pr(\text{Test positive}) = \Pr(D=1|H=0) \Pr(H=0) + \Pr(D=1|H=1) \Pr(H=1) = 0.01 \cdot 0.9985 + 1 \cdot 0.0015 = 0.011485$$

Hence we get $\Pr(H = 1|D = 1) = \frac{\Pr(D=1|H=1) \Pr(H=1)}{\Pr(D=1)} = \frac{1 \cdot 0.0015}{0.011485} = 0.131$, in other words, there's only a 13.1% chance that the patient actually has AIDS, despite using a test that is 99% accurate! As we can see, statistics can be quite counterintuitive. 

## Cardiac Segmentation

The [original](https://github.com/zackchase/mxnet-the-straight-dope/blob/master/chapter01_crashcourse/probability.ipynb) version of this tutorial implemented a Naive Bayes Classifier to classify digits from the MNIST dataset. We recommend you review that version as well for additional insights. In this example we will work with simulated cardiac MRI cine data to classify individual pixels, otherwise known as pixelwise classificiation (i.e. segmentation).

The data consists of MRI cine images in the short-axis view and the corresponding segmentations. The size of each image is 80x80 and there are 600 samples. The segmentations consist of 4 labels: 0-background, 1-heart tissue, 2-blood pool, and 3-body activity. Let us load and split the data into training and testing samples:

In [9]:
datapath = joinpath(pwd(), "../datasets/cardiac_data.h5");
nsamples = 600;
nlabels  = 4;
size     = 80;
data     = Array{Float64}(nsamples, size, size);
labels   = Array{Int64}(nsamples, size, size);

for sample_id = 1:nsamples
    Data = h5read(datapath, @sprintf "sample_%d" sample_id);
    x, y = Data;
    data[sample_id, :, :]   = x.second;
    labels[sample_id, :, :] = y.second + 1; # add one to avoid label 0
end

test = 0.10;
r    = randperm(nsamples);
n    = round(Int, (1 - test) * nsamples);
xtrn = data[r[1:n], :, :];
ytrn = labels[r[1:n], :, :];
xtst = data[r[n+1:end], :, :];
ytst = labels[r[n+1:end], :, :];

Now, let's look at some of the training samples:

In [10]:
hms = [heatmap(xtrn[i, :, :], color=:gray, legend=false) for i in 100:100:400];
plot(hms..., layout = (2,2), colorbar = false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
<image width="251" height="164" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAPsAAACkCAYAAABRqliEAAAgAElEQVR4nO2dx38kx5GFoz3cGLgx
GI4BzZAiKemk3+7/r92bdNklZUhJJMcbjDcwje7eAxeV34vOqKkGuHtgZZxyGlVZVVk1Ge7Fi06v
250ZhP+YzeRPH/y90+lkf/d/i86PrtHtdsNzmxwX/c57mk6nje6F50TzNnm+unl/SeE9TiaT//Pr
RcLr+WvXrf2JNPnmFv3GcveSO7/pe+JxvV7vg/dSd79ck+iaW5ub1fg3X/6mGt/+7HY1Xl1brcbp
SyhSpMivWsp/9iJFWiL9SQMTyqzeDDuRLs3iGnM7+r3OXI9+7+GcaWBuzfCMTdwU/7foeSPzMzLP
+Hz+/CYmaHRPdc/xS5ruTdye07xDuYZeMHvtJnP535t8v9G74rPOfX+Buc7xou/ZTP8vRd/s0729
arz5+Ek1vnnjZjUejUZpzvBqRYoU+VVJ3+9yZwkaRZq16byDwaAaHx0dZY+ZmxfjaDeWc4Ld1Gvd
aGeOrJfomZru5E00ZST+2qcJVC16bpPAWJM18f+W9xZcm8f0+/1qTCuGGtefs6jVVnf8ooFZnl13
dPS36H6p5e8/uF+Nz507V42LZi9SpCXS97tRpB2b+GKUOp8pkkibL3Kd3O/c5aMdu6kWox/ZbaAV
OjXH84qL+qR1smhqtEmap25e0Xy/4LzRMVE68axr1STWUpf6pTSJEdgpLOFoTZ8/f16N7929V40v
X7pcjYtmL1KkJTLns1uDHS3yO5qCVJqAXETrcq4afy+6Bnf/SJpq9tNExE/EZyjO4lufJrayKLij
Lo5BEW2Dc3hXTS3DRpHyQGtOcIx/52fJEtRZgJEGb6KNzwq+in5/+epVNT48PKzGRbMXKdISmdPs
UeRZhDtYMPFZfTTZwT549Lw08RfrtGO0MzeJ+J9V057FevDS5F6a5Ifr7ovSRIs1jeeEfnqDdW/q
A0f3FcV5vETfic8G5KTO2ogsU867tbWVvY9rOzvVuOTZixRpoXQ6nU6oLhbVKtx16vzkReetO35h
rXAKRFmdH/uh4xufy3Pw82niBdFxTda9aTFTpMFFuzXQxnXX/L+KY/Bv/GaPj48/eI2meXZZdx6P
8fr6ejVeggY2Mxsfj6txt5PWkdp851rS4OsX01zdXjr+0val9LsVKVKkFdLp1mj2s2qYM91YAy1S
d06EjZdoMY6p29WbIMEW9evn7vEMa3cabRNZO3XWx1nqG+oyNWfB7J9Gm1MWjbvUXePC+fPpOKzD
ACi/9Y2Nary9nbT0xYsXw3mHg2E17g/SXBcuXKjGqyuplHWG/7kd2BVFsxcp0hLpdD15xRnQTE0j
roviy5tikyM5jVWyqH/b5JimGm3RvGpdJLeRTxn4lx4XEN37ou+g7m9R5LtJRVrTaPxZxM97/fr1
avzZp59WY5JGLC8tV2Pe74WLSTOfW0sYdjOzg4ODdM1uuma/lzS7Zigm2WP4QotmL1KkJTKPoINE
u2GMrCOGfG6yNPyF/Kezymnq2SNs/aLaNPfv00qTDIE/LrTCGloM4bvC8Ty7aexi0XtvGsmPvidG
xDlTD342ZWNjXf5969atary7u1uNpxNU5sHPPjpM9R9LS0vVmNF3M7PhKPnpJq8nQA8eQ7Pj3jku
mr1IkZbIXNVbJI18NPEPHDKvwVyLor2aHhcho5pWvZ0lb01piuqKjmkaHwnnDfDl0Z3X3W8TrHlT
rRv56aurye9dW1urxvv7++l3HMPf++BGMNOc/yrmunnzRvZ68s100zczWtJ8OKPgzIf3hsjfj1Om
hzlw8bnHiksZT5Om7wdWBl8cLYFBPz374VHBxhcp0jppHI1fVMM0rZhqkquu05pN/MgI789Z/TWa
YqNz12tqrTTFD+SOiZBqZk5TzzhcLKtw1szHMnzS5ZWVcF5aHCvQrru7t6rxGqPVQYyhX0PffDxJ
2nV5OUXHmatewT2Ox0mz0v8eOIshWlP+TByD+OIQr73pg/OarGJbWk7rS21Oo5pR/aLZixRpidRi
42tOqsZnzV8u6gee5r6a3K+v8KvLMefmbZIrrtPeq6tJqwyHyS+MsPxcq15P92xqojGQgbRWoqh5
t2at+G9qG16D2O11oMLOX0joMj6fmdN8mJdz8XmZd2YGiL/TfzZTzR7lqulPHx4kDcpoumhQJ3wO
amq+N/rQFObizcyms8CapFGD551N0x/4/dGSKJq9SJGWSL+Oy/w0bCi5c72cJaLt5awtmKrjGx63
6P0ylzpylU3UXPQdN4Gfpgrm7h3FKrzImuB3nt8PKr/mItrQfIwwc97RMGkS5rD5HPSZzdQPpVA7
UjPPjmFhQNPymXh/Zk4L4o4jH5javBYFiXkjzc5zaBHR+iACzkwzAPoO85ml4XJa97dv36Zrw8or
mr1IkZZIx5xSO4vffBq02C9ZKRddb1FGmKbzMnpLjUbMM/PDvZ5GXHn+EvK3K8vpd0abe4E295Hc
MPMR+Hh857xfHu/njXK5gtiC5qIPSq1lZtbrp3/zXt6/T3lz+vLUbvS/qR29z85nkdhFJ1gTWDFE
vVHje+H7pMVAbU7Lib9TA/98X2k8CbJUUaZFvgccUzR7kSItkTmffdGqt6YYdsHxLsgWcxr8/qKV
Y3XXoI+5AT90+1JiAaE/Llzm2JW9v8fdf4pzXoEdlJqLGQLO63HcUb6Z50+hBUej5KuSu7/rovxL
OK67iih2J28NdnpUT2novznmtDkXtbmWXeQj6PKeXRSG70Si/8N83ps15JSOxdYOEXxjoubwTOLL
Q5v7++UzMu7D9yNzBZaTYEzmnqZIkSK/SjlVPXskjX32AAEVaf9wHidNOm9GFWL0Vc00R7x9absa
L8Of5s56gF19/2AfvycE01wlFn1o4pzpn+IPjEhz957LqMzovzG3nq53jPUlk0qk6cy0b9gAx62s
JMuHWuj8uZRbPxonjeQ/Md4jLZkouxL5wNR6g6FmEoSxhRiFfp43UdhhAg1aN+/BYXrvPJ/+P3H2
PnvA98BYBOMCtCQYQ6GVUDR7kSItlDNr9ibcXv64s+LsKWepmrt27Vo1vrS9LccxBy47M7QHfet3
795l71eew3GJ824jX4wiePg+59LnprYThJjkhIGygqbUyK/O24f2YESbmYiVVWYYksZfhUbyWpdW
CrV25INLpD145T66LRF4Phc+d/F7A973OQy7WEiIrguHfJ5l12cMKFwj4Qjs5K0SZhK4JoU3vkiR
FkqtZl8ULXaaqrdFo+N1DCTRcaxT3tzcrMaXL6doOrtdmmm1EH2jN0AnUYMeBxh0qg5Gs83M3r1P
1gC15njMbrao6oJWYTTeI90Y5afWjuIYoq1Okfmgz0+cPFGBjIksuZrwzY30TqjF+N7k3gMtT6ug
riMwYxq0OPh8vA/61vS5zRweP7BAQ6s20NJmMY5hIMg8/N/BffBbZOyhaPYiRVoitVVvi3LQRceY
na2T5Wn89ytXrlTjy5eT1l5fT1F27ozHY+UAo3aktnoP35w7KNlEufsP4TN5pdnBzj6GJuIuz3wt
143aeOy0WMRBRm0lVX3U8oEl4OeaBZpWMhSoHCOjDDW+mUMcrgJxiLkGgzwyj3n2yLc1i9lXeb5E
wAMsiI+a89kjyyLKSghvnEfmMZaAtSbmg88knHfBOy+avUiRlsgcu2wjDu+gw

### Naive Bayes classification

Conditional independence is useful when dealing with data, since it simplifies a lot of equations. A popular algorithm is the Naive Bayes Classifier. The key assumption in it is that the attributes are all independent of each other, given the labels. In other words, we have:

$$p(x|y) = \prod_i p(x_i|y)$$

For images, $x$ refers to a single image which has many pixel attributes, i.e. $x\in \mathbb{R}^n$, where $n=80\times80$. In this case $p(x_i,y)$ is the probability of observing a pixel intensity $x_i$ at pixel $i$ if the label is $y$ for the image. However, $p(x|y)$ assumes a single label for the entire image. For image segmentation we have to deal with labels for each individual pixel. Thus, we need not worry about independence between attributes since we are triying to find

\begin{equation}
p(x_i|y)\,\,\text{for}\,\,i=1\dots n
\end{equation}

only in this case $y$ is defined for every pixel. Using Bayes Theorem this leads to the classifier

\begin{equation}
p(y|x_i) = \frac{p(x_i|y) p(y)}{p(x_i)}
\end{equation}


where $x_i$ is a measured intensity, $p(y|x_i)$ is the posterior probability, $\ p(x_i|y)$ is the measurement model, and $p(y)$ is the prior. The measurement model characterizes the sensor by saying:given a tissue class $y$ (e.g. $y$=cardiac tissue), what is signal intensity distribution?

Unfortunately, this is still intractable, since we don't know $p(x)$. Fortunately, we don't need it, since we know that $\sum_y p(y|x) = 1$, hence we can always recover the normalization from $p(y|x) \propto p(x_i|y) p(y)$. After all that math, it's time for some code to show how to use a Naive Bayes classifier for segmentation of the cardiac dataset. 

The problem is that we don't actually know the prior $p(y)$ or $p(x_i|y)$. So we need to *estimate* it given some training data first. This is what is called *training* the model. In the case of 4 possible classes we simply compute $n_y$, i.e. the number of occurrences of class $y$ and then divide it by the total number of occurrences. Likewise, to get an idea of $p(x_i|y)$ we add the intensities of pixel $i$ when set for diagnosis $y$ and then divide it by the sum of the intensities for with that label (as opposed to dividing for the number of occurrences of diagnosis $y$ when dealing with digits. This is the tissue class conditional model of signal intensity. 


In [11]:
ycount = ones(nlabels);
xcount = ones(size * size, nlabels);
for sample_id = 1:n
    x = reshape(xtrn[sample_id, :, :], (size * size, ))
    y = reshape(ytrn[sample_id, :, :], (size * size, ))
    for pixel = 1:size*size
        ycount[y[pixel]] += 1
        xcount[pixel, y[pixel]] += x[pixel]
    end
end

for label_id = 1:nlabels
    xcount[:, label_id] /= sum(xcount[:, label_id])
end

py = ycount / sum(ycount)

4-element Array{Float64,1}:
 0.278753
 0.156232
 0.268572
 0.296443

Now that we computed per-pixel counts of occurrence for all pixels (pixelwise), it's time to see how our model behaves. Time to plot it. We show the estimated probabilities (based on signal intensities) pixelwise. These are some mean looking cardiac tissues.

In [12]:
labels_name = Dict(1=>"background", 2=>"myocadium_act", 3=>"bloodpool_act", 4=>"body activity");

In [13]:
hms = [heatmap(reshape(xcount[:, i], (size, size)), title=labels_name[i], legend=false, aspect_ratio=:equal) for i in 1:4];
print(py)
plot(hms..., layout = (2,2), colorbar = false)

[0.278753, 0.156232, 0.268572, 0.296443]

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 background 
 
 
<image width="144" height="144" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAJAAAACQCAYAAADnRuK4AAAgAElEQVR4nO19WY+k2ZnWc74lvlgz
cq/K2rqqul12t8fLgGHEDAOaAYHmDq64gAEEFyBxw5/hH3DBDUJCGg2jQWjkRcxgt91dttvurr0q
KysyK5fYI771cPE+74mMbC/jCdudZZ3nJjMivuV8y3nOu7/m4I+/YnEO2TwBADTaEwCAtQHyeQ0A
UGvOAQBBUAEAqiqQbSqDPJVt4iTnkeSwJpC/ZR4h43EanamcaybnCkI5XpRkyPldY30k58gjOQ63
mQ/biJJMvjNy7ILblPwb13KAv+mxqzLkmA3HmSFupPJdIb9l0wYAIIwKdz/KIjp/e1By2yjO3e+W
xwwiPZfcl6Q1AwCk46Ybs45Dz6HHi2s5ogvjKbOY97By9yCb1gEANW6r91Dvc/vqMSzvA/jduLcF
AOifbAAArt7Zx4sHtwEAe7deLt27/vEmAOB4sI63rstvT17cBAA0Ejnnnz+9I9dVGgTw8FgBURCV
AADLWVNrkGX4fVhL3QzXtz2M5TdDlrEwi5kVy6yrigU7AUBUy90stGSuKOZMJ1vAGsc0OWcaIPtb
2RWtrTNkE2GKIo+XjhOfY6aSvwWc6TqLQx51dLqOhNeaKLOGcl1zHj9pzdz4dX+9P9lMxwc0umP5
Tsds5fpSHieq5e44yi76OWlP3ffW8l7VZaZP+x0ZM59FY2OIkKxUZNHSb2t3hS2KURPpsAVgwZ7K
5jUy4mzQ5l0Forrcs+57T2Wf73wBANBqTzAcrAGAYyJl/Hf7wlJPhh3PQB6rISr1Ta7JTE3HTQCL
mRLlOWLOCGUOnWnt7b47kJUJivGZvLU6YyMyQBBWTk7qXjuScw3bcjwyW1xP3YwKKWOoXBLV5POs
v+ZksbhBuYJsp6xT5JFjskplGJKczsrOxsDJcDquMpdr1pk2G7bR6Iy5jYxRWUK3sZVxbK33SX8L
OeYirbltlLmsVQaR+xPVM0xOunJ/bhwCAJpkjvlIGCUbN1EjY8UtuQcVn196vC6f8xAx78/8SJhC
ZbKC19naHODW9hkAYHCwuzRWfW794y3UybbXfu++3I9nVwAAv032LB98wTOQx2qI0rnMiO7aCYDF
7HHaRpI7LWw2lpkQUlbQdTyfJYi5ll7UelpbwlJVESGYyH6TY9EGVJPQmVaVoTtXOhEma6wJA+js
CeMCKRkw1hl+XvsCgDiHyk7KLirfqBwmYyLjUFtJyQ7dnVOOobG4UWTEhbbJ4xhgRllFWdLaZaYu
ixhhmS9tU+9MlsaQTRro7J5yYLK/smjnijybdNR0smH7vVcAgP77ohE1doRRqjR2LBlSNnQy0Lrc
y9nhJo4e31y6Z3pc1aJ3r/XQ3pPzHv7VewCA9bfknFe++EjO/cMvewbyWA3m9D+9bQGgoP1HZQSV
8muN1M0SN9PJNoXTCGKnnSiKbLHeArI2qwylM0TZSs9ljIVTDyiz6NqsMlo2aTiWqlMe0JmqTJKN
Gyh4HUlTNA+VpXTmZ7PEse1izPHS+FrrI3fNKqvMlYX5ubk5wIxy38KGJuNSO0zcnGF0uLV0foVq
lHEjRUL7WD6hrWdNPqt8aQxQ2xrK/eQ9mx1tuN8AoL7dRzGWY/b3RWbRZ6ArxsnBFXevVEbd/fwT
OS4ZNhs3cbS/BwDY3BEmGpNp3/qn3wUAHH7jPUQqeOqDUsEyPmf4UqEs4TIwORGBTYXZqJYDSpsq
jPHhpCMRyssicsJcsyuUqsuSCphlvjDKWb5JOr4ol/Hk88QtR7rfnGqrLh15FqOl5+BL0eBnNZhV
ReSEXv1OJ097s8/Pi5dex6oTRZfofNpw41i7egwZSMBtAjfOxtpk6TtVFjbe2Zd9jEXYkvEkVAr0
JVHDYDGtw/K37EweZvvtAwDA8Me3AMgLpQZIfVn7h9sAgCvvPgYAXOtOkPZFgalxXBkVmsMnN2R8
YYlr9+Slat4RoX7+jS/LULmyb737xC9hHqvBPPsXX7PAwj0RUYhy6n1cwpjl33Q26jozHbZRp5FK
obNbl6QyX8x4NRUolPWSxhyTgcwsnambt3oAgHyaLM7tlkAaNLmE6fdxa4aM5wid8Cv7O3PAoO0E
dFWtlYma61wmisgJ7LrcnlcKACBMMse2uvREzXRpPPm0jtZNmcVG2ZbXE8SLawjast/s6Q6AhdBb
ztVdUTnXTs6lTz+rqr/5znOEHXkWxZAmGX1efMbTo013nzsbA46D5gQqNq8+uY2U92xzS4T79dvC
dhFNCNHvbngG8lgN5ujfv2eBhfk852yccwY3OpOFw6+uctGymlrksWORJme1QgXwqgyc2V4FWmU9
J/yeMweoEO2MdJS3snFj4UTlWq/mgIXMUbkDOAWAjKHnDILKCfoq16iKnlB2MIF159JxKNTpGzfm
UJpVdlE5RT/Xb5y46ylORdYIKO9MX4ghL6qnbmYHvAd6nMm+bBPEhVMmCt77kNuGlFnTfgc1jl8N
tXoPJgP5nM7raLaogPD61PGrhuCqCrB5XVhT5dqH998FAHzlj/9CLmYOz0Aeq8FoOIczkKk5XkMB
zrpY25Y10Jxzep6Hso/8Jn9KaimWf2uNuZMnVPaYnHWXjmOtQZ0zQ8ej2oo6RcO4+JTGN2aYQnhu
G5XhnIrvrti6c6kWqdd10Yja2B44rTDj7G1efy3XnDLUIioBbhOu0ZE8ITt9iZraRzmsMjH3yyiD
NN8SGc8EFuVY7s+U4RcJDYCqhWXj5qdcKYPX4q5QDevwo7uY0Tjc4r18fSzHC/hMm/UZdq4Ju+Rk
cWX+Jo2WZ49uoEVX1bMfvSPHa8rxNqhtrv9hzzOQx2qIogtONLWFBJCZ3+yMnbHLaUs0zunnIKww
OhU22b0rdg0NKchmtHskGSrKVTOuzRfdFHlac5qewrkZOAZhMvm/uyczoUPXg2oitjKoaIvZuE4t
jvurLJOfc1O0yAI601UGMa0SSGXG1++Jq8DEtNEMaTg1QNChrWifTKGGxW/KPU12U2foCxJhydo1
0X7KUxmH6cw/xZaqfdWvyXWaqESp98qFtMg5D77/OQDimrh6U7QltVXNKdfe/t0PAACDhzfRoNuk
RqNsSY0r4P0/OtwFDkX26rS5Ykzl+eUvxMCY/UkC0/u3v2XPn0xfKCegtmYY03C4dkUuZMqlR5eZ
jeuHmPAFcjcqXHjhFfqAVRUe0yem/qk4yVzMigrzKhjrUph0Jo7G1X+jL7JbirCwaF/0jGv8S+P2
4eJJ0WFvqO1aauHlacOpxBXVbqMveKUXahE0GcP0SiaGjq/2Fo2ZvTqiPb5wvB3Zc1nC9IGZsHJC
twrW49cbvAcaa1U4gV8nlotX4nWOTtaxcf2IFyTfHdLvtUGLcljL8fFHEvcz4Ytz+6r4ucYTeUav
R1289/ZDAMDZiSyT3XV56a9+7Udy+N+57pcwj9UQqaleZ/GI7LLmDGahcz0MaRJXz3SbRqjJyfrC
E62RdvTrqNuj0R07VVFN7RpJqBGBZRFiSibT46lLRZewWb/zqdnXpK8npDofdWZOmI+2OGPXhV7s
BpnyEG4bx0QJt5nLOaPfrsOUjGwMGcs4l+PZtjCI6feBmGx9nffy2RDnEbbnKI9jnkvHLtue/vi2
nLo1c78p4zuXT6ZLfOx8cTp2vQQ93nDUAQ44VLL4674w

Now we can compute the likelihoods of a pixel, given the model. This is statistician speak for $p(x|y)$, i.e. how likely it is to see a particular image under certain conditions (such as the label). Since this is computationally awkward (we might have to multiply many small numbers if many pixels have a small probability of occurring), we are better off computing its logarithm instead.  That is, instead of $p(x|y) = \prod_{i} p(x_i|y)$ we compute $\log p(x|y) = \sum_i \log p(x_i|y)$. 

$$l_y := \sum_i \log p(x_i|y) = \sum_i x_i \log p(x_i = 1|y) + (1-x_i) \log \left(1-p(x_i=1|y)\right)$$

To avoid recomputing logarithms all the time, we precompute them for all pixels. 

In [14]:
logxcount    = log.(xcount);
logxcountneg = log.(1-xcount);
logpy        = log.(py);

Pred = zeros(nsamples-n, size, size)
for sample_id = 1:nsamples - n
    x = reshape(xtst[sample_id, :, :], (size * size, ))
    y = reshape(ytst[sample_id, :, :], (size * size, ))
    
    logpx = zeros(size*size, nlabels)
    for label_id = 1:nlabels
        logpx[:, label_id] += py[label_id] + logxcount[:, label_id].*x + logxcountneg[:, label_id].*(1-x)
    end

    
    logpx -= maximum(logpx)
    px  = exp.(logpx)
    px /= sum(px)

    pred = reshape(mapslices(indmax, px, 2), (size, size))
    Pred[sample_id, :, :] += pred
    
end

Let us evaluate the total accuracy:

In [15]:
total_accuracy = sum(Pred .== ytst) / ((nsamples-n) * size * size)

0.7772838541666667

Now, let us evaluate the predicted segmentations. Feel free to try different subject ids i, specially for cases with low individual accuracy (displayed on the title):

In [16]:
i   = 5
p   = [Pred[i, :, :], ytst[i, :, :]]
acc = sum(Pred[i, :, :] .== ytst[i, :, :]) / (size * size)
t = [@sprintf("prediction accuracy %f",acc), "true labels"]
hms = [heatmap(p[j], legend=false, color=:lightrainbow, title=t[j]) for j in 1:2];
plot(hms..., layout = (1,2), colorbar = false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 prediction accuracy 0.840469 
 
 
<image width="251" height="344" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAPsAAAFYCAYAAACVjYpSAAAHKElEQVR4nO3dPW4cxxaAUdJwoL0o
VKolUOtw5tQKGUqpMq9DW3BIhtqLMzowILd+hqaome6q+s6J+ATDb2zjw8Wtrh5e393cPVwBy/vl
6A8A7EPsECF2iBA7RIgdIsQOEWKHCLFDhNghQuwQIXaIEDtEiB0ixA4RYocIsUOE2CFC7BAhdogQ
O0SIHSLEDhFihwixQ4TYIULsECF2iBA7RIgdIsQOEWKHCLFDhNghQuwQIXaIEDtEiB0ixA4RYocI
sUOE2CFC7BAhdogQO0SIHSLEDhFihwixQ4TYIULsECF2iBA7RIgdIsQOEWKHCLFDhNghQuwQIXaI
EDtE/Hr0B4C9vbq6PfojPNv95rOf+ue4P/HnJjtEiB0ixA4R13c3dw9Hfwj4npl361Fs93eTHSLE
DhFihwjP2Sdy+/Lvoz/Cd91+evHdP3/qc2C7+T5MdogQO0SIHSLEDhEu1Qxo1IO4c/n46d3RHyHJ
ZIcIsUOE2CHCzr6j1Xfxc7DPX47JDhFihwixQ4QXYS7Abs6ITHaIEDtEiB0iPGc/E3v6ZXn+/vNM
dogQO0SIHSLEDhEu1fwgB3HMymSHCLFDhNghwqWaJ7Cnj8UFm+cx2SFC7BAhdoiws59gT5+D/f3p
THaIEDtEiB0ixA4R6RdhHMJRYrJDhNghQuwQkd7Zmd+bl28//+yCzeNMdogQO0SIHSJyO7tn61SZ
7BAhdogQO0SIHSIS31Sz+qHcH79f5jLJ+w9v//8vGpQLNt8y2SFC7BAhdohY9lLNinv6pXbz5/z/
zbzPV5nsECF2iBA7RCz7nH2VnX3vPf05ZtjfPXc32SFD7BAhdohY6jn7Knv6bLbnCjPs71UmO0SI
HSLEDhFih4ilDuhmNsPlmadwWDcukx0ixA4RYoeI6V+EWeUizSo7+ykj7e/Vl2JMdogQO0SIHSI8
Zz/Q6ns6YzHZIULsECF2iBA7RDig21H5QM4LMscz2SFC7BAhdoiYbmef4cWX8m7OuEx2iBA7RIgd
Iqb78oqjd3b7+Pkd+dy99EUWJjtEiB0ixA4RYoeI6S7V7MVB3H68JLMPkx0ixA4RYoeI6Xb2208v
vvzfZ7pkY0dndSY7RIgdIsQOEVPs7Nu9/OudHXgakx0ixA4RYoeIOXb2C+3pnq1TYrJDhNghQuwQ
IXaImOKAbuuc3y67/aIEh3WszmSHCLFDhNghYrqdHc7pzcv/zm1W/+0wJjtEiB0ixA4RU+zse/zm
1q9/OYHn7vvxiyH2YbJDhNghQuwQIXaIGOqA7tXV7eef7zc/s5ZRD+RWv2BjskOE2CFC7BBxfXdz
93D0h/ieU/v7HhdsvuaCzc8bdU8/xc4OTEvsECF2iBjqOfuW5+zzm21P31rxmbvJDhFihwixQ4TY
IWKKSzXbw5KjuWDzuJkP5bZWOZTbMtkhQuwQIXaImGJn3xppf98q7/Kr7OmnrLK/m+wQIXaIEDtE
DPsizGy2e+uq+/vqu/kpq7wUY7JDhNghQuwQYWe/gFO77Qy7fHUvv7r68r/Piv8eTHaIEDtEiB0i
xA4RQ70Ic+rll8eM+mIMY7nU4ejr3y7yt70Ikx0ixA4RYoeI6S/VbF9MsL+3HXFpaaaXZEx2iBA7
RIgdIqbf2ekZ6YWiLz7L4M/cTXaIEDtEiB0ixA4R078Ic4oLNmsZ6VDuKUZ8QcZkhwixQ4TYIWLZ
SzVekJnTbLv5TEx2iBA7RIgdIoba2e83z9nP+cydsdjLj2GyQ4TYIULsECF2iFj2RZhTXLDZj4O4
f43yUozJDhFihwixQ8RQl2r24AWZ87Obz8FkhwixQ4TYIeLwnf3IF17s74+zi6/FZIcIsUOE2CFi
qLvxW0d/ecUqO7y9+3juxgO7EjtEiB0ixA4Rhx/QHX0Q9xSnDuscfvGjjjysM9khQuwQIXaIOPxF
mBnYzVmByQ4RYocIsUOE2CFC7BAhdogQO0SIHSLEDhFihwixQ4TYIULsECF2iBA7RIgdIg758ooZ
vmRya/slgX/9edzngJ9hskOE2CFC7BAhdog45IDufnNAN9thHczKZIcIsUOE2CFit519u5vfT7yn
u2DDrEx2iBA7RIgdIrwIAxEmO0SIHSLEDhFn3dnt4jAukx0ixA4RYocIsUPEIZdqVuGlGGZiskOE
2CFC7BDxrJ3d5Zlv2d855f2Ht59/fn317rDPYbJDhNghQuwQ8ejObjeH59nu6VtvXv735x8/7bu/
m+wQIXaIEDtEiB0irh9ubh6O/hArc8FmbacO4k7Z+1Buy2SHCLFDhNghwpdXXJgXZNbyozv6SEx2
iBA7RIgdIjxnP5Adflzn3M2PfLa+ZbJDhNghQuwQIXaIcEA3CId1x7jUJZlRDuW2THaIEDtEiB0i
7OwDsr+f3x4vsIy4p2+Z7BAhdogQO0TY2Qdnf3+e0vPzpzLZIULsECF2iPgHtvpSfx9AswkAAAAA
SUVORK5CYII=
" transform="translate(29, 31)"/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 true labels 
 
 
<image width="251" height="344" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAPsAAAFYCAYAAACVjYpSAAAHm0lEQVR4nO3dMXLk1hVAUVKl3TjU
NqitOJXDCeVUW9E2OKHWQwejKcOU0UI3GsAH7jkxS+oh69ar94EGXt/f3j9egFP76eXL3/7MD9t/
DGAEYocIsUOE2CFC7BAhdogQO0SIHSJ+PPoDAOt9ndxUM3eDjckOEWKHCLFDhNghQuwQIXaIEDtE
iB0ixA4RYocIsUOE2CFC7BAhdogQO0SIHSI8vAIuZu5BFiY7RIgdIsQOEWKHCLFDhNghQuwQIXaI
EDtEiB0ixA4RYocIsUOE2CFC7BAhdojw8Aq4gOlDKuaY7BAhdogQO0SIHSIc0MGAlhy43ctkhwix
Q4TYIcLODgPYYkf/zGSHCLFDhNghQuwQIXaIEDtEiB0ixA4RbqohbY+bWUZhskOE2CFC7BBhZ+cU
Srv1Vkx2iBA7RIgdIuzsF7fVrvt18t+1T5+DyQ4RYocIsUOE2CHCAd2JjHQQNtJnYRmTHSLEDhFi
hwg7+4Du3Yd//se/tvkgD/j9j1+P/gjMMNkhQuwQIXaIeH1/e/84+kNUnXk3X8NefwyTHSLEDhFi
hwg7+8aW7uVX2cfXss9vx2SHCLFDhNghQuwQ4YBuAw7lnsNh3XOZ7BAhdogQO0TY2VdYspvby5/P
Lv8Ykx0ixA4RYocIO/ud5vZ0u/kx7O/LmewQIXaIEDtEiB0iHNAtMOKh3C//HOdg6t+/jXE46bDu
NpMdIsQOEWKHCG9xnTHKnj63m9/ak/fe5x/5jOzPZIcIsUOE2CHCdfY/3XoQxR57+h5770jX5qe2
2O1dc/8rkx0ixA4RYocIsUNE+qaapW9u2cr0wOwqN6A88u/Y4vfw+VDVgZ3JDhlihwixQ0R6Z79l
jxtprrKnrz

## Sampling

Random numbers are just one form of random variables, and since computers are particularly good with numbers, pretty much everything else in code ultimately gets converted to numbers anyway. One of the basic tools needed to generate random numbers is to sample from a distribution. In Julia we use `rand` to pick a random element or array of random elements:

In [17]:
rand()

0.7402995315425946

In [18]:
rand(1,3)

1×3 Array{Float64,2}:
 0.937207  0.0981407  0.00208214

In [19]:
rand(3,1)

3×1 Array{Float64,2}:
 0.0811156
 0.898507 
 0.321449 

Let's see what happens when we use a random number generator:

In [20]:
for i = 1:10; display(rand()); end

0.23502183135609656

0.7788837626512832

0.06297533244102849

0.6552707431085483

0.15386713769952287

0.39591654965163636

0.9127621475153116

0.7789635126655632

0.8304669632201809

0.6220554623886443

### Uniform Distribution

These are some pretty random numbers. As we can see, their range is between 0 and 1, and they are evenly distributed. That is, there is (actually, should be, since this is not a *real* random number generator) no interval in which numbers are more likely than in any other. In other words, the chances of any of these numbers to fall into the interval, say $[0.2,0.3)$ are as high as in the interval $[.593264, .693264)$. Random number generation in Julia uses the Mersenne Twister library via MersenneTwister objects. The way they are generated internally is to produce a random integer first, and then divide it by its maximum range. If we want to have integers directly, try the following instead. It generates random numbers between 0 and 100.

In [21]:
for i = 1:10; display(rand(0:100)); end

16

100

58

0

4

42

8

5

89

77

Note that we can specify the type or an indexable collection to pick from. What if we wanted to check that ``rand`` is actually really uniform. Intuitively the best strategy would be to run it, say 1 million times, count how many times it generates each one of the values and to ensure that the result is uniform. 

In [22]:
hms = [histogram(rand(1:100, n), bin=:100, legend=false) for n in [10, 100, 1000, 10000, 100000, 1000000]];
plot(hms..., layout = (2,3), colorbar = false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 15 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 0 
 
 
 250 
 
 
 500 
 
 
 750 
 
 
 1000 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip7706)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 304.621,211.966 30

What we can see from the above figures is that the initial number of counts looks *very* uneven. If we sample fewer than 100 draws from a distribution with over 100 possible outcomes this is pretty much expected. But even for 1000 samples there is a significant variability between the draws. What we are really aiming for is a situation where the probability of drawing a number $x$ is given by $p(x)$. 

### The categorical distribution

Quite obviously, drawing from a uniform distribution over a set of 100 outcomes is quite simple. But what if we have nonuniform probabilities? Let's start with a simple case, a biased coin which comes up heads with probability 0.35 and tails with probability 0.65. Suppose that we generate a uniform random variable over $[0,1]$ and if the number is less than $0.35$, we output heads and otherwise we generate tails. What is the probability of getting heads? since this is a uniform distribution where every value is equally likely, the probability is simply the sum of all individual probability between 0 and 0.35, or 35%. Let's try this out. 

First we use `rand(n)` to generate $n$ uniform random variables. Then we ask which of these tosses were heads by checking pointwise .< 0.35. We take the cumsum to keep track of how many heads we have accumulated and then we divided by the number of tosses as we go. Let us plot this scheme:

In [23]:
n = 1000;
plot(cumsum(rand(n) .< 0.35)  ./ [1:n;], title="Predicted probability of Heads", labels=:"Estimated P(d=Heads)")
hline!([0.35], color="black", line=(:dot,4), labels=:"True Probability")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 200 
 
 
 400 
 
 
 600 
 
 
 800 
 
 
 1000 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 0.4 
 
 
 0.5 
 
 
 Predicted probability of Heads 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Estimated P(d=Heads) 
 
 
 
 True Probability

As we can see, on average this sampler will generate 35% zeros and 65% ones. We can generalize this idea to multiple outcomes as follow: Given any probability distribution, e.g. 
$p = [0.1, 0.2, 0.05, 0.3, 0.25, 0.1]$ we can compute its cumulative distribution (Julia's ``cumsum`` will do this for you) $F = [0.1, 0.3, 0.35, 0.65, 0.9, 1]$. Once we have this we draw a random variable $x$ from the uniform distribution $U[0,1]$ and then find the interval where $F[i-1] \leq x < F[i]$. We then return $i$ as the sample. By construction, the chances of hitting interval $[F[i-1], F[i])$ has probability $p(i)$. Let's see how this works:

First we define our probabilities $p$ and take the cumsum. We then we use `rand(n)` to generate $n$ uniform random variables and check which values satisfy `F .< rand(n)`:

In [7]:
srand(2)
p = KnetArray([0.1,0.2,0.05,0.3,0.25,0.1]);
F = prefixsum(p)'

LoadError: [91mUndefVarError: prefixsum not defined[39m

In [5]:
Knet.gpu(0)

0

In [25]:
r = rand()

0.36679641243992434

In [26]:
F .<= rand()

1×6 RowVector{Bool,BitArray{1}}:
 true  true  true  false  false  false

As we can see, r lies between index 3 and 4, therefore indices 3 and 4 are the only ones that satisfy $F[i-1] \leq x < F[i]$. Indeed, there is a 30% chance of getting this interval since this is the lenght of the interval itself, therefore we need to sample index 4. How might we do this efficiently for multiple samples at once? consider that it's always true that the value we desire is the first `false` value, and it occurs at the index one after the last `true` value. Thus, we may sum all the values and add one to get the desired index!  Let's see how this works for multiple samples:

In [27]:
sum(F .<= rand(4), 2) + 1

4×1 Array{Int64,2}:
 2
 5
 4
 4

ok, we are now ready to visualize this for multiple samples, this time we normalize the histogram:

In [28]:
hms = [histogram((sum(F .<= rand(n), 2) + 1), bin=:6, legend=false, normalize=true) for n in [4, 20, 40, 400, 4000, 10000]];
plot(hms..., layout = (2,3), colorbar = false)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 4.00 
 
 
 4.25 
 
 
 4.50 
 
 
 4.75 
 
 
 5.00 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 0.0 
 
 
 0.1 
 
 
 0.2 
 
 
 0.3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
 
 5 
 
 
 6 
 
 
 7 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25

Notice that it doesn't take to many samples to converge to the true distribution.

### The Normal distribution

The Normal distribution (aka the Gaussian distribution) is given by $p(x) = \frac{1}{\sqrt{2 \pi}} \exp\left(-\frac{1}{2} x^2\right)$. Let's plot it to get a feel for it.

In [29]:
x = -10:0.01:10
p = (1 / sqrt(2pi)) * exp.(-0.5x.^2)
plot(x, p)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -10 
 
 
 -5 
 
 
 0 
 
 
 5 
 
 
 10 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 0.20 
 
 
 0.25 
 
 
 0.30 
 
 
 0.35 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip8302)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none"

Sampling from this distribution is a lot less trivial. First off, the support is infinite, that is, for any $x$ the density $p(x)$ is positive. Secondly, the density is nonuniform. There are many tricks for sampling from it - the key idea in all algorithms is to stratify $p(x)$ in such a way as to map it to the uniform distribution $U[0,1]$. One way to do this is with the probability integral transform. 

Denote by $F(x) = \int_{-\infty}^x p(z) dz$ the cumulative distribution function (CDF) of $p$. This is in a way the continuous version of the cumulative sum that we used previously. In the same way we can now define the inverse map $F^{-1}(\xi)$, where $\xi$ is drawn uniformly. Unlike previously where we needed to find the correct interval for the vector $F$ (i.e. for the piecewise constant function), we now invert the function $F(x)$. 

In practice, this is slightly more tricky since inverting the CDF is hard in the case of a Gaussian. It turns out that the *twodimensional* integral is much easier to deal with, thus yielding two normal random variables than one, albeit at the price of two uniformly distributed ones. For now, suffice it to say that there are built-in algorithms to address this. 

The normal distribution has yet another desirable property. In a way all distributions converge to it, if we only average over a sufficiently large number of draws from any other distribution. To understand this in a bit more detail, we need to introduce three important things: expected values, means and variances. 

* The expected value $\mathbb{E}_{x \sim p(x)}[f(x)]$ of a function $f$ under a distribution $p$ is given by the integral $\int_x p(x) f(x) dx$. That is, we average over all possible outcomes, as given by $p$. 
* A particularly important expected value is that for the function $f(x) = x$, i.e. $\mu := \mathbb{E}_{x \sim p(x)}[x]$. It provides us with some idea about the typical values of $x$.
* Another important quantity is the variance, i.e. the typical deviation from the mean 
$\sigma^2 := \mathbb{E}_{x \sim p(x)}[(x-\mu)^2]$. Simple math shows (check it as an exercise) that
$\sigma^2 = \mathbb{E}_{x \sim p(x)}[x^2] - \mathbb{E}^2_{x \sim p(x)}[x]$.

The above allows us to change both mean and variance of random variables. Quite obviously for some random variable $x$ with mean $\mu$, the random variable $x + c$ has mean $\mu + c$. Moreover, $\gamma x$ has the variance $\gamma^2 \sigma^2$. Applying this to the normal distribution we see that one with mean $\mu$ and variance $\sigma^2$ has the form $p(x) = \frac{1}{\sqrt{2 \sigma^2 \pi}} \exp\left(-\frac{1}{2 \sigma^2} (x-\mu)^2\right)$. Note the scaling factor $\frac{1}{\sigma}$ - it arises from the fact that if we stretch the distribution by $\sigma$, we need to lower it by $\frac{1}{\sigma}$ to retain the same probability mass (i.e. the weight under the distribution always needs to integrate out to 1). 

Now we are ready to state one of the most fundamental theorems in statistics, the [Central Limit Theorem](https://en.wikipedia.org/wiki/Central_limit_theorem). It states that for sufficiently well-behaved random variables, in particular random variables with well-defined mean and variance, the sum tends toward a normal distribution. To get some idea, let's repeat the experiment described in the beginning, but now using random variables with integer values of $\{0, 1, 2\}$. 